# Evaluation of Cross-modal Query Suggestion methods on CroQS benchmark

In [1]:
device_str = "cuda:0"
device_str_llm = device_str

import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from lib.CroQS import CroQS

croQS_json_path = "./CroQS_Benchmark_v1.0.0.json"
croQS = CroQS(croQS_json_path)
device = "cuda:0"
hdf5_file_path = os.getenv("HDF5_INDEX_FILE_PATH")
croQS.evaluation_init(hdf5_file_path, device)

File ./dump.json not found
No self.queries dict object loaded from disk, creating new empty dict


In [3]:
len(croQS.list_queries())

50

In [4]:
from lib.methodsLoader import get_qe_methods_dict, add_method_to_dict

qe_methods_names = [
    'initial-query',
    #'clipcap-on-centroids', 
    #'decap-on-centroids', 
    ##'clipcap-query_set-of-embeddings', 
    #'decap-on-representatives', 
    #'clipcap-on-representatives',
    #'clipcap-prompting-on-centroids',
    #'clipcap-prompting-on-representatives',
    ]

In [36]:
def load_qe_dict(ir_system, load_group_cap : bool = False):
    from lib.methods.decap.Im2TxtProjection import Im2TxtProjector, ProjectionType
    from lib.methods.decap.DecapQueryExpansion import DeCapQueryExpansion
    from lib.methods.decap.decap import get_decap_model
    from lib.methods.groupcap.model import GroupCapLLM, LLMBasePrompts, ImageCaptioningMethods, LLMTypes

    decap_qe = DeCapQueryExpansion.load_object(device_str, 'coco')
    assert isinstance(decap_qe, DeCapQueryExpansion)
    decap_qe.im2txt.ir_system = ir_system
    qe_dict = get_qe_methods_dict(methods_keys=qe_methods_names, decap_qe=decap_qe, device=device_str)

    GC_TYPE = "llama3-8b" # "gemma2-2b" # "llama3-8b", "mistral7b"

    if load_group_cap:

        if GC_TYPE == "gemma2b":
            llm_type = LLMTypes.Gemma2b_IT
        elif GC_TYPE == "gemma2-2b":
            llm_type = LLMTypes.Gemma2_2b
        elif GC_TYPE == "mistral7b":
            llm_type = LLMTypes.Mistral7b_IT
        elif GC_TYPE == "llama3-8b":
            llm_type = LLMTypes.LLama3_8b_IT
        else:
            print(f"Using default LLM - LLama3-8b")
            GC_TYPE = "llama3-8b"
            llm_type = LLMTypes.LLama3_8b_IT

        group_cap_obj = GroupCapLLM(ImageCaptioningMethods.decap, decap_qe.get_generated_captions, llm_type, device_map=device_str_llm, image_captioning_method_device=device_str)
        
        prompts_dict = GroupCapLLM.get_prompts_dict()

        group_cap_obj.qe_prompt = prompts_dict['query-expansion']['qe-few-shot-4']
        qe_dict = add_method_to_dict(qe_dict, f'groupcap-{GC_TYPE}-qe-few-shot-4', group_cap_obj.generate_expanded_query, wants_query=True, method_group_name="img-embeddings-whole-cluster")

    return qe_dict

qe_dict = load_qe_dict(croQS.ir_system, True)

In [38]:
import pandas as pd

q = "italy"
r = croQS.get_query_suggestions_dataframe_for_query(q, qe_dict)

r[0]

Loading already existing qs for the method 'initial-query'

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '0' of query = 'italy'
Loading already existing qs for the method 'initial-query'

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '1' of query = 'italy'
Loading already existing qs for the method 'initial-query'

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '3' of query = 'italy'
Loading already existing qs for the method 'initial-query'

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '4' of query = 'italy'


,query,cluster,method,suggested-query,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,italy,0,initial-query,italy,1.000,0.264,1.000,0.129875,0.215520,0.527778,0.222222
1,italy,0,groupcap-llama3-8b-qe-few-shot-2,Italy,1.000,0.264,1.000,0.129875,0.215520,0.527778,0.222222
2,italy,0,human,italy streets,0.500,0.286,0.904,0.870125,0.724558,0.861111,0.833333
3,italy,0,groupcap-llama3-8b-qe-few-shot-3,italy,1.000,0.264,1.000,0.129875,0.215520,0.527778,0.222222
4,italy,0,groupcap-llama3-8b-qe-few-shot-4,Italy,1.000,0.264,1.000,0.129875,0.215520,0.527778,0.222222
5,italy,1,initial-query,italy,1.000,0.263,1.000,0.371854,0.347826,0.402439,0.292683
6,italy,1,groupcap-llama3-8b-qe-few-shot-2,italy,1.000,0.263,1.000,0.371854,0.347826,0.402439,0.292683
7,italy,1,human,italy pizza,0.500,0.310,0.878,0.921602,0.792000,0.743902,0.987805
8,italy,1,groupcap-llama3-8b-qe-few-shot-3,italy,1.000,0.263,1.000,0.371854,0.347826,0.402439,0.292683
9,italy,1,groupcap-llama3-8b-qe-few-shot-4,italy with pizza,0.500,0.306,0.873,0.694765,0.696532,0.658537,0.987805


In [39]:
ret_df = croQS.get_query_suggestions_dataframe_all_queries(qe_dict)

ret_df

Loading already existing qs for the method 'initial-query'3-8b-qe-few-shot-4'

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '0' of query = 'person reading'
Loading already existing qs for the method 'initial-query'

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '1' of query = 'person reading'
Loading already existing qs for the method 'initial-query'

...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '5' of query = 'bridge'
Loading already existing qs for the method 'initial-query'

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '6' of query = 'bridge'


,query,cluster,method,suggested-query,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,dog running,0,initial-query,dog running,1.000,0.293,1.000,0.408536,0.497725,0.548387,0.720430
1,dog running,0,groupcap-llama3-8b-qe-few-shot-2,dog running outdoors,0.667,0.287,0.963,0.588467,0.594678,0.591398,0.827957
2,dog running,0,human,dog running catching a frisbee,0.500,0.319,0.830,0.864315,0.741513,0.580645,0.892473
3,dog running,0,groupcap-llama3-8b-qe-few-shot-3,dog running with a frisbee,0.667,0.319,0.792,0.860382,0.648007,0.526882,0.913978
4,dog running,0,groupcap-llama3-8b-qe-few-shot-4,dog running to catch frisbee,0.500,0.311,0.872,0.933746,0.812702,0.612903,0.881720
...,...,...,...,...,...,...,...,...,...,...,...
1470,bridge,6,initial-query,bridge,1.000,0.270,1.000,0.138862,0.213246,0.606061,0.181818
1471,bridge,6,groupcap-llama3-8b-qe-few-shot-2,bridge over a river,0.500,0.269,0.938,0.224006,0.293281,0.727273,0.272727
1472,bridge,6,human,bridge over a river near forest,0.250,0.266,0.897,0.599694,0.456155,0.666667,0.363636
1473,bridge,6,groupcap-llama3-8b-qe-few-shot-3,bridge over a river,0.500,0.269,0.938,0.224006,0.293281,0.727273,0.272727


In [30]:
ret_df = croQS.get_query_suggestions_dataframe_all_queries(qe_dict)

ret_df

Loading already existing qs for the method 'groupcap-llama3-8b-qe-few-shot-3'

,query,cluster,method,suggested-query,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,dog running,0,initial-query,dog running,1.000,0.293,1.000,0.408536,0.497725,0.548387,0.720430
1,dog running,0,groupcap-llama3-8b-qe-few-shot-2,dog running outdoors,0.667,0.287,0.963,0.588467,0.594678,0.591398,0.827957
2,dog running,0,human,dog running catching a frisbee,0.500,0.319,0.830,0.864315,0.741513,0.580645,0.892473
3,dog running,0,groupcap-llama3-8b-qe-few-shot-3,dog running with a frisbee,0.667,0.319,0.792,0.860382,0.648007,0.526882,0.913978
4,dog running,2,initial-query,dog running,1.000,0.294,1.000,0.215018,0.214729,0.500000,0.294118
...,...,...,...,...,...,...,...,...,...,...,...
1175,bridge,5,groupcap-llama3-8b-qe-few-shot-3,bridge near a river,0.333,0.259,0.924,0.000000,0.122222,0.481481,0.111111
1176,bridge,6,initial-query,bridge,1.000,0.270,1.000,0.138862,0.213246,0.606061,0.181818
1177,bridge,6,groupcap-llama3-8b-qe-few-shot-2,bridge over a river,0.500,0.269,0.938,0.224006,0.293281,0.727273,0.272727
1178,bridge,6,human,bridge over a river near forest,0.250,0.266,0.897,0.599694,0.456155,0.666667,0.363636


In [40]:
error_type = 'std'

aggregated_df_2 = ret_df.groupby(['query', 'method']).mean().reset_index()
title_info = "Macro Averaged "

score_cols = list(aggregated_df_2.columns)
for el in ['query','method']: score_cols.remove(el)

# Group by method and compute mean and error for each score column
agg_dict = {col: ['mean', error_type] for col in score_cols}
grouped_2 = aggregated_df_2.groupby('method').agg(agg_dict).reset_index()
grouped_2

/tmp/ipykernel_987741/2624988023.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_df_2 = ret_df.groupby(['query', 'method']).mean().reset_index()


method jaccard-q            \
                                         mean       std   
0  groupcap-llama3-8b-qe-few-shot-2  0.659922  0.157349   
1  groupcap-llama3-8b-qe-few-shot-3  0.668567  0.169887   
2  groupcap-llama3-8b-qe-few-shot-4  0.551813  0.130300   
3                             human  0.437972  0.112474   
4                     initial-query  1.000000  0.000000   

  clip-similarity-cluster           clip-similarity-q             NDCG@10  \
                     mean       std              mean       std      mean   
0                0.283306  0.018148          0.939840  0.029049  0.277522   
1                0.284210  0.018974          0.927612  0.047191  0.309838   
2                0.288767  0.017386          0.900695  0.037484  0.353558   
3                0.298296  0.015991          0.870624  0.032796  0.518537   
4                0.279894  0.018076          1.000000  0.000000  0.177377   

                  MAP           Recall-Open-Set-@100            \
        std      mean       std                 mean       std   
0  0.144467  0.275342  0.112867             0.481073  0.113584   
1  0.178642  0.293558  0.135260             0.495029  0.109589   
2  0.155959  0.325102  0.127053             0.486121  0.127187   
3  0.139276  0.445149  0.108550             0.620265  0.121844   
4  0.053571  0.210269  0.053614             0.518257  0.032359   

  Recall-Closed-Set            
               mean       std  
0          0.314199  0.136502  
1          0.336709  0.169050  
2          0.400165  0.167331  
3          0.591542  0.126442  
4          0.190265  0.071740

In [6]:
ret_df = croQS.get_query_suggestions_dataframe_all_queries(qe_dict)

ret_df

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/home/giacomo.pacini/miniconda3/envs/py38/lib/python3.8/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


Computing scores for cluster '0' of query = 'dog running'


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '2' of query = 'dog running'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '5' of query = 'bridge'


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Computing scores for cluster '6' of query = 'bridge'


,query,cluster,method,suggested-query,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,dog running,0,initial-query,dog running,1.000,0.293,1.000,0.408536,0.497725,0.548387,0.720430
1,dog running,0,groupcap-llama3-8b-qe-few-shot-2,dog running outdoors,0.667,0.287,0.963,0.588467,0.594678,0.591398,0.827957
2,dog running,0,human,dog running catching a frisbee,0.500,0.319,0.830,0.864315,0.741513,0.580645,0.892473
3,dog running,2,initial-query,dog running,1.000,0.294,1.000,0.215018,0.214729,0.500000,0.294118
4,dog running,2,groupcap-llama3-8b-qe-few-shot-2,dog running outdoors,0.667,0.277,0.963,0.000000,0.061329,0.176471,0.088235
...,...,...,...,...,...,...,...,...,...,...,...
880,bridge,5,groupcap-llama3-8b-qe-few-shot-2,bridge near a river,0.333,0.259,0.924,0.000000,0.122222,0.481481,0.111111
881,bridge,5,human,train over a bridge,0.500,0.301,0.879,0.580602,0.568275,0.703704,0.555556
882,bridge,6,initial-query,bridge,1.000,0.270,1.000,0.138862,0.213246,0.606061,0.181818
883,bridge,6,groupcap-llama3-8b-qe-few-shot-2,bridge over a river,0.500,0.269,0.938,0.224006,0.293281,0.727273,0.272727


### Micro averaged results

In [7]:
grouped_df = ret_df.groupby(by='method').mean(numeric_only=True).reset_index()#.sort_values(by=sort_by_column, ascending=False)

grouped_df

,method,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,groupcap-llama3-8b-qe-few-shot-2,0.654441,0.284122,0.939064,0.253999,0.254895,0.473312,0.290618
1,human,0.437193,0.299312,0.870451,0.497983,0.430224,0.626201,0.571772
2,initial-query,1.000000,0.280369,1.000000,0.162076,0.194837,0.515198,0.170162


### Macro averaged results

In [8]:
import pandas as pd

tmp_df = None

for q in ret_df['query'].unique().tolist():
    df_q = ret_df[ret_df['query'] == q]
    grouped_df_q = df_q[ grouped_df.columns.to_list() + ['query', 'cluster', 'suggested-query'] ].groupby(by='method').mean(numeric_only=True).reset_index()#.sort_values(by=sort_by_column, ascending=False)
    
    if tmp_df is None:
        tmp_df = grouped_df_q
    else:
        tmp_df = pd.concat([tmp_df, grouped_df_q], ignore_index=True)

macro_averaged_df = tmp_df.groupby(by='method').mean(numeric_only=True).reset_index()#.sort_values(by=sort_by_column, ascending=False)
macro_averaged_df

,method,jaccard-q,clip-similarity-cluster,clip-similarity-q,NDCG@10,MAP,Recall-Open-Set-@100,Recall-Closed-Set
0,groupcap-llama3-8b-qe-few-shot-2,0.659922,0.283306,0.939840,0.277522,0.275342,0.481073,0.314199
1,human,0.437972,0.298296,0.870624,0.518537,0.445149,0.620265,0.591542
2,initial-query,1.000000,0.279894,1.000000,0.177377,0.210269,0.518257,0.190265


In [12]:
ret_df.columns

Index(['query', 'cluster', 'method', 'suggested-query', 'jaccard-q',
       'clip-similarity-cluster', 'clip-similarity-q', 'NDCG@10', 'MAP',
       'Recall-Open-Set-@100', 'Recall-Closed-Set'],
      dtype='object')

In [16]:
error_type = 'std'

aggregated_df = ret_df.groupby(['query', 'method']).mean().reset_index()
title_info = "Macro Averaged "

score_cols = list(aggregated_df.columns)
for el in ['query','method']: score_cols.remove(el)
score_cols

/tmp/ipykernel_987741/3485062696.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_df = ret_df.groupby(['query', 'method']).mean().reset_index()


['jaccard-q',
 'clip-similarity-cluster',
 'clip-similarity-q',
 'NDCG@10',
 'MAP',
 'Recall-Open-Set-@100',
 'Recall-Closed-Set']

In [17]:
error_type = 'std'

aggregated_df = ret_df.groupby(['query', 'method']).mean().reset_index()
title_info = "Macro Averaged "

score_cols = list(aggregated_df.columns)
for el in ['query','method']: score_cols.remove(el)

# Group by method and compute mean and error for each score column
agg_dict = {col: ['mean', error_type] for col in score_cols}
grouped = aggregated_df.groupby('method').agg(agg_dict).reset_index()
grouped

method jaccard-q            \
                                         mean       std   
0  groupcap-llama3-8b-qe-few-shot-2  0.659922  0.157349   
1                             human  0.437972  0.112474   
2                     initial-query  1.000000  0.000000   

  clip-similarity-cluster           clip-similarity-q             NDCG@10  \
                     mean       std              mean       std      mean   
0                0.283306  0.018148          0.939840  0.029049  0.277522   
1                0.298296  0.015991          0.870624  0.032796  0.518537   
2                0.279894  0.018076          1.000000  0.000000  0.177377   

                  MAP           Recall-Open-Set-@100            \
        std      mean       std                 mean       std   
0  0.144467  0.275342  0.112867             0.481073  0.113584   
1  0.139276  0.445149  0.108550             0.620265  0.121844   
2  0.053571  0.210269  0.053614             0.518257  0.032359   

  Recall-Closed-Set            
               mean       std  
0          0.314199  0.136502  
1          0.591542  0.126442  
2          0.190265  0.071740

In [9]:
qq = 'person at the phone'
cc = '3'

croQS._list_clusters_labels(qq)

['0', '1', '2', '3']

In [10]:
croQS.suggested_queries[qq][cc]

{'suggested-queries': {'initial-query': {'query': 'person at the phone',
   'scores': {'jaccard-q': 1.0,
    'clip-similarity-cluster': 0.294,
    'clip-similarity-q': 1.0,
    'NDCG@10': 0.23365080824310536,
    'MAP': 0.2868582289545055,
    'Recall-Open-Set-@100': 0.5128205128205128,
    'Recall-Closed-Set': 0.28205128205128205}},
  'groupcap-llama3-8b-qe-few-shot-2': {'query': 'person talking on a cell phone',
   'scores': {'jaccard-q': 0.5,
    'clip-similarity-cluster': 0.291,
    'clip-similarity-q': 0.934,
    'NDCG@10': 0.1100458831490401,
    'MAP': 0.13931386869090717,
    'Recall-Open-Set-@100': 0.358974358974359,
    'Recall-Closed-Set': 0.3076923076923077}},
  'human': {'query': 'woman talking on mobile phone',
   'scores': {'jaccard-q': 0.2,
    'clip-similarity-cluster': 0.317,
    'clip-similarity-q': 0.877,
    'NDCG@10': 0.39639187290150935,
    'MAP': 0.3438731991303363,
    'Recall-Open-Set-@100': 0.5128205128205128,
    'Recall-Closed-Set': 0.5897435897435898}}}}